<a href="https://colab.research.google.com/github/ja-errorpro/Google-Build-with-AI-2025-materials/blob/main/GDG_On_Campus_CYCU_Build_with_AI_2025_Discord_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDG On Campus CYCU - Build with AI 2025

這份筆記將展示如何將 Gemini AI 串接到 Discord Bot


Made By YiJia Huang
- GDG On Campus CYCU 2025 Organizer
- [![Personal Website](https://img.shields.io/badge/Personal%20Website-green?style=flat-square&link=https://ja-errorpro.codes)](https://ja-errorpro.codes)
- [![GitHub](https://img.shields.io/badge/GitHub-Follow%20me-blue?style=social&logo=github&link=https://ja-errorpro.codes)](https://ja-errorpro.codes)
- [![LinkedIn](https://img.shields.io/badge/LinkedIn-0A66C2?style=flat-square&link=https://www.linkedin.com/in/乙家-黃-a95b7a114/)](https://www.linkedin.com/in/乙家-黃-a95b7a114/)

## Prerequisites

您可以在 Google Colab 中運行此教程，無需額外的環境配置。

**由於 Colab 的 async 機制與資料安全，**

**如果你想要在本地端執行，請使用此這份 Repo 中的程式碼: [Gemini Bootcamp with Discord Bot](https://github.com/ja-errorpro/Gemini-Bootcamp-with-Discord-bot)，內容與 Colab 上的程式碼略有不同**

## Install the SDK

安裝 Gemini API Python SDK [`google-generativeai`](https://pypi.org/project/google-generativeai/)

In [ ]:
!pip install -q -U google-generativeai

## Install Discord Python SDK

安裝 [discord-py-interactions](https://pypi.org/project/discord-py-interactions/) 來串接 Discord API

In [ ]:
!pip install -q -U discord-py-interactions

## 載入必要的套件

In [ ]:
import interactions
from interactions import slash_command, listen, SlashContext
import asyncio
import nest_asyncio
# 允許 Google Colab 運行 asyncio 事件循環
nest_asyncio.apply()
import google.generativeai as genai
# 此套件將用來保護你的密鑰
from google.colab import userdata

## 初始化聊天機器人

請登入 [Discord 開發者網站](https://discord.com/developers/applications)，創建機器人，並取得 Token

若要使用 Gemini API，您需要一個 API 密鑰。如果您還沒有密鑰，可以在 Google AI Studio 中創建一個密鑰。

[Get an API key](https://aistudio.google.com/app/apikey)

在 Colab 中，將 Token 與密鑰添加到左側面板中的 "🔑" `Secret`。

將 Discord Bot Token 命名為 DISCORD_BOT_TOKEN。

將 Gemini API 密鑰命名為 GOOGLE_API_KEY。

然後將這些資訊傳遞給 SDK:

In [ ]:
# 設定要給機器人監聽的事件
# 範例開啟了所有監聽事件
intents = interactions.Intents.ALL
bot = interactions.Client(intents=intents)

DISCORD_TOKEN = userdata.get('DISCORD_BOT_TOKEN')

## 初始化生成模型

In [ ]:
GENAI_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GENAI_API_KEY)

## 調整 Gemini 設定

這裡可以調整模型的行為參數

舉例來說，Temperature 將決定機器人回應的創意程度

In [ ]:
generation_config = {
   "temperature": 1,
   "top_p": 0.95,
   "top_k": 40,
   "max_output_tokens": 1024, # 注意：由於 Discord 訊息的字數限制，請不要設超過 2000
   "response_mime_type": "text/plain"
}

## 設定系統提示(System Instruction)

這是讓 AI 能夠正確運作的重要部分。

這將定義 AI 的角色、行為和任務，並確保 AI 能夠符合使用者的要求。

範例的系統提示並不完整也不一定是最好的，請根據您的需求進行修改。

Example:
你是一名 Google Developer Groups On Campus 的核心成員，專注於協助學生使用 Flutter 以及 GCP 進行開發，擁有豐富的專案開發經驗。

你可以參考這個連結來了解更多：
https://github.com/google-gemini/cookbook/blob/main/quickstarts/System_instructions.ipynb

In [ ]:
system_prompt = [
    "你是一名 AI 助理，",
    "你喜歡且會盡可能地幫助使用者解決任何程式開發相關的問題。",
    "你全程都必須使用繁體中文溝通",
]

## 輸出可用的模型

- gemini-1.5-flash：最快的多模態模型
- gemini-1.5-pro：最強大的智慧多模態模型

你可以自行決定要使用哪個版本的模型，

注意：若要進一步了解頻率限制和模型功能，請參考 [Gemini 模型](https://ai.google.dev/gemini-api/docs/models/gemini?hl=zh-tw)

此段程式碼來源：(https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


參考上方程式碼的輸出，請填入你想使用的模型

範例使用 `gemini-1.5-pro`

In [ ]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    system_instruction=system_prompt,
    generation_config=generation_config,
)

## 定義機器人行為

In [ ]:
@bot.event
async def on_ready(): # 當機器人準備好時，輸出 Logged in as 機器人名稱
  print(f'Logged in as {bot.user}', flush=True)

## 定義斜線指令

根據使用者的 prompt 生成回應

當機器人上線時，將向 Discord 註冊指令

使用者需要在聊天室使用：/ask \<prompt\>

來觸發指令

In [ ]:
@slash_command(name = "ask",
       description="與機器人聊天",
       options=[
        interactions.SlashCommandOption( # 定義指令選項
        name="prompt", # 選項名稱，注意：必須與下方定義函數的參數名稱相同，如果你想使用不同的參數名稱，請在 @slash_command() 裡使用 argument_name 參數
        description="輸入你想要對機器人說的話", # 選項的描述
        type=3, # 這個參數型別是字串
        required=True # True 表示這個參數必填
        )
       ]
) # 新增一個叫做 ask 的 command
async def chat(ctx: SlashContext, prompt: str):
  # 因為 Discord Bot 若沒有回應超過五秒就會 Timeout，因此需要使用 defer() 告訴 Discord 需要等待
  await ctx.defer()
  # 設定使用者的 prompt
  content = [
    {
      "role": "user",
      "parts": prompt
    }
  ]
  try:
    response = model.generate_content(content)
    if response.text:
      await ctx.send(response.text)
    else:
      await ctx.send("目前機器人無法回應，請稍後再試")
  except Exception as e:
    await ctx.send("目前機器人無法回應，請稍後再試: " + str(e))

  return

## 啟動機器人

注意：如果你上次已經執行過此機器人，請重新初始化一次機器人以更新 Session

In [ ]:
async def start_bot():
  bot.start(DISCORD_TOKEN)

asyncio.run(start_bot())